In [21]:
using LinearAlgebra
using Random
rng = MersenneTwister(5);

Random.seed!(rng, 69)

n = 4;

G = zeros(Float64,n,n)
    for i=1:n    
    G[i,i] = rand(rng, 1:5)                 # positive diagonals
    G[i+1:n, i] = rand(rng, -10:10, n-i)    # G is lower triangular
end

A = G * G'
display(A)
display(G)

for eigval in eigvals(A)
    @assert eigval > 0
end

x_orig = rand(rng, 0:10, n)
b = A * x_orig
display(x_orig)
display(b)

4×4 Matrix{Float64}:
 25.0   0.0   45.0  10.0
  0.0   1.0   -5.0  -7.0
 45.0  -5.0  122.0  37.0
 10.0  -7.0   37.0  73.0

4×4 Matrix{Float64}:
 5.0   0.0   0.0  0.0
 0.0   1.0   0.0  0.0
 9.0  -5.0   4.0  0.0
 2.0  -7.0  -4.0  2.0

4-element Vector{Int64}:
 7
 0
 6
 0

4-element Vector{Float64}:
  445.0
  -30.0
 1047.0
  292.0

In [18]:
# Cholesky decomposition, num flops = 1/3 n^3 (classic LU is 2/3 n^3, LU with pivoting is n^3)

function factorize_cholesky(A)
    display("Original Matrix")
    display(A)
    n = size(A, 1)
    for j=1:n
        for k=1:j-1
            for i=j:n
                A[i,j] = A[i,j] - A[i,k] * A[j,k]
            end
        end

        sqrt_ajj = sqrt(A[j,j])

        for i=j:n
            A[i,j] /= sqrt_ajj
        end
        display(j)
        display(A)
    end    
    return A
end
;

In [19]:
A_fact = factorize_cholesky(A)

L = tril(A_fact)
@assert norm(L - G) == 0        # lower triangular part of A now contains G, the original matrix used to compute A

"Original Matrix"

4×4 Matrix{Float64}:
 25.0   0.0   45.0  10.0
  0.0   1.0   -5.0  -7.0
 45.0  -5.0  122.0  37.0
 10.0  -7.0   37.0  73.0

1

4×4 Matrix{Float64}:
 5.0   0.0   45.0  10.0
 0.0   1.0   -5.0  -7.0
 9.0  -5.0  122.0  37.0
 2.0  -7.0   37.0  73.0

2

4×4 Matrix{Float64}:
 5.0   0.0   45.0  10.0
 0.0   1.0   -5.0  -7.0
 9.0  -5.0  122.0  37.0
 2.0  -7.0   37.0  73.0

3

4×4 Matrix{Float64}:
 5.0   0.0  45.0  10.0
 0.0   1.0  -5.0  -7.0
 9.0  -5.0   4.0  37.0
 2.0  -7.0  -4.0  73.0

4

4×4 Matrix{Float64}:
 5.0   0.0  45.0  10.0
 0.0   1.0  -5.0  -7.0
 9.0  -5.0   4.0  37.0
 2.0  -7.0  -4.0   2.0

In [20]:
# function to solve for x, using cholesky factorized A

function solve(A_fact, b)
    x = copy(b)
    
    for j=1:n
        x[j] /= A_fact[j,j]
        for i=j+1:n
            x[i] -= A_fact[i,j] * x[j]
        end
    end
    
    for i=n:-1:1
        for j=i+1:n
            x[i] -= A_fact[j,i] * x[j]
        end
        x[i] /= A_fact[i,i]
    end
    return x
end

x_sol = solve(A_fact, b)
@assert norm(x_sol - x_orig) == 0